In [1]:
import json
import pandas as pd
import csv
from datetime import date
from datetime import datetime
import time
import matplotlib.pyplot as plt
import os
import nltk
from nltk.corpus import twitter_samples, stopwords
'''
nltk.download('averaged_perceptron_tagger')
nltk.download("stopwords")
nltk.download('wordnet')
nltk.download('twitter_samples')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Daphne/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Daphne/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Daphne/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     /Users/Daphne/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Daphne/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [24]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier

import re, string, random

def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

if __name__ == "__main__":

    positive_tweets = twitter_samples.strings('positive_tweets.json')
    negative_tweets = twitter_samples.strings('negative_tweets.json')
    text = twitter_samples.strings('tweets.20150430-223406.json')
    tweet_tokens = twitter_samples.tokenized('positive_tweets.json')[0]

    stop_words = stopwords.words('english')

    positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
    negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

    positive_cleaned_tokens_list = []
    negative_cleaned_tokens_list = []

    for tokens in positive_tweet_tokens:
        positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    for tokens in negative_tweet_tokens:
        negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    all_pos_words = get_all_words(positive_cleaned_tokens_list)

    freq_dist_pos = FreqDist(all_pos_words)
    print(freq_dist_pos.most_common(10))

    positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
    negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

    positive_dataset = [(tweet_dict, "Positive")
                         for tweet_dict in positive_tokens_for_model]

    negative_dataset = [(tweet_dict, "Negative")
                         for tweet_dict in negative_tokens_for_model]

    dataset = positive_dataset + negative_dataset

    random.shuffle(dataset)

    train_data = dataset[:7000]
    test_data = dataset[7000:]

    classifier = NaiveBayesClassifier.train(train_data)

    print("Accuracy is:", classify.accuracy(classifier, test_data))

    print(classifier.show_most_informative_features(10))

    custom_tweet = "I ordered just once from TerribleCo, they screwed up, never used the app again."

    custom_tokens = remove_noise(word_tokenize(custom_tweet))

    print(custom_tweet, classifier.classify(dict([token, True] for token in custom_tokens)))

[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]
Accuracy is: 0.996
Most Informative Features
                      :( = True           Negati : Positi =   2055.5 : 1.0
                follower = True           Positi : Negati =     21.9 : 1.0
                     sad = True           Negati : Positi =     20.0 : 1.0
               community = True           Positi : Negati =     17.1 : 1.0
                    blog = True           Positi : Negati =     16.4 : 1.0
                    damn = True           Negati : Positi =     15.6 : 1.0
               goodnight = True           Positi : Negati =     13.8 : 1.0
                    glad = True           Positi : Negati =     13.5 : 1.0
                     ugh = True           Negati : Positi =     12.2 : 1.0
                follback = True           Positi : Negati =     11.7 : 1.0
None
I ordered just once from TerribleCo, they screwed 

In [3]:
tweets_df = pd.read_csv("results.csv", index_col=0, header=0, engine='python')


In [4]:
tweets_df = tweets_df.drop(columns = ["Tweet Id"])

In [5]:
tweets_df = tweets_df[tweets_df["Username"]!= "jeonginree"]

In [6]:
tweets_df = tweets_df.dropna()

In [7]:
tweets_df1 = tweets_df[tweets_df["Text"].str.contains('@Crypto_Guy_UK')==False]

In [8]:
from langdetect import detect
def is_en(txt):
    try:
        return detect(x) =='en'
    except:
        return False

#tweets_df1[tweets_df1['Text'].apply(is_en)]
tweets_df2 = tweets_df1[tweets_df1.Text.map(lambda x: x.isascii())]

In [9]:
tweets_df2

,Datetime,Text,Username
0,2021-02-28 23:59:58+00:00,@runako Anything Mercedes and probably Tesla b...,jasminexETH
2,2021-02-28 23:59:49+00:00,Whats happened in #Bitcoin since the #Tesla pu...,ToneVays
3,2021-02-28 23:59:47+00:00,Had to blackout the Queen #yhlqmdlg #TeslaMod...,Scarlett3401
4,2021-02-28 23:59:40+00:00,@ValaAfshar @Tesla @jayleno @elonmusk @elonmus...,elonHCMCmusk
7,2021-02-28 23:59:29+00:00,@Hitecsoftouch @Energy_Tidbits @JustinTrudeau ...,Davideilers67
8,2021-02-28 23:59:18+00:00,pressing my nipple to the fingerprint lock on ...,trojan__whores
9,2021-02-28 23:59:17+00:00,Mark your friends who should benefit from an a...,Qwertycoin_QWC
11,2021-02-28 23:59:06+00:00,@ElectrekCo @FredericLambert Tesla needs a dop...,Jermaine__HD
12,2021-02-28 23:59:02+00:00,Hey @yunggravy did Alexa ever answer how many ...,JustTylerJett
14,2021-02-28 23:58:49+00:00,@PatrickMcGee_ @schlaf @a16z @zipline *Skydio ...,bigsantry


In [10]:
tweets_df2.to_csv('cleaned.csv')

In [2]:
tweets_df = pd.read_csv("cleaned.csv", index_col=0, header=0, engine='python')

In [20]:
#tweets_df["Text"].str.count(" ")

In [25]:
tweets_df1 = tweets_df[tweets_df["Text"].str.count(" ") > 1]

In [26]:
sentiment = []
for custom_tweet in tweets_df1.loc[:,"Text"]:
  custom_tokens = remove_noise(word_tokenize(custom_tweet))
  sentiment.append(classifier.classify(dict([token, True] for token in custom_tokens)))
tweets_df1["sentiment"] = sentiment

In [28]:
tweets_df1.to_csv("tweet_sentiment.csv")